In [2]:
"""
Some sources: 

- https://github.com/wobine/blackboard101/blob/master/EllipticCurvesPart4-PrivateKeyToPublicKey.py
- https://wiki.openssl.org/index.php/Elliptic_Curve_Cryptography
- https://onyb.gitbook.io/secp256k1-python/point-addition-in-python


Most of the code below copied from https://onyb.gitbook.io/secp256k1-python/point-addition-in-python, check it out!

"""
from dataclasses import dataclass
import hashlib
from typing import Tuple

@dataclass
class PrimeGaloisField:
    prime: int

    def __contains__(self, field_value: "FieldElement") -> bool:
        # called whenever you do: <FieldElement> in <PrimeGaloisField>
        return 0 <= field_value.value < self.prime
    
@dataclass
class FieldElement:
    value: int
    field: PrimeGaloisField

    def __repr__(self):
        return "0x" + f"{self.value:x}".zfill(64)
        
    @property
    def P(self) -> int:
        return self.field.prime
    
    def __add__(self, other: "FieldElement") -> "FieldElement":
        return FieldElement(
            value=(self.value + other.value) % self.P,
            field=self.field
        )
    
    def __sub__(self, other: "FieldElement") -> "FieldElement":
        return FieldElement(
            value=(self.value - other.value) % self.P,
            field=self.field
        )

    def __rmul__(self, scalar: int) -> "FieldValue":
        return FieldElement(
            value=(self.value * scalar) % self.P,
            field=self.field
        )

    def __mul__(self, other: "FieldElement") -> "FieldElement":
        return FieldElement(
            value=(self.value * other.value) % self.P,
            field=self.field
        )
        
    def __pow__(self, exponent: int) -> "FieldElement":
        return FieldElement(
            value=pow(self.value, exponent, self.P),
            field=self.field
        )

    def __truediv__(self, other: "FieldElement") -> "FieldElement":
        other_inv = other ** -1
        return self * other_inv


@dataclass
class EllipticCurve:
    a: int
    b: int

    field: PrimeGaloisField
    
    def __contains__(self, point: "Point") -> bool:
        x, y = point.x, point.y
        return y ** 2 == x ** 3 + self.a * x + self.b

    def __post_init__(self):
        # Encapsulate int parameters in FieldElement
        self.a = FieldElement(self.a, self.field)
        self.b = FieldElement(self.b, self.field)
    
        # Check for membership of curve parameters in the field.
        if self.a not in self.field or self.b not in self.field:
            raise ValueError

P: int = (
    0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
)
field = PrimeGaloisField(prime=P)

# Elliptic curve parameters A and B of the curve : y² = x³ Ax + B
A: int = 0
B: int = 7

secp256k1 = EllipticCurve(
    a=A,
    b=B,
    field=field
)


inf = float("inf")

@dataclass
class Point:
    x: int
    y: int

    curve: EllipticCurve

    def __post_init__(self):
        # Ignore validation for I
        if self.x is None and self.y is None:
            return

        # Encapsulate int coordinates in FieldElement
        self.x = FieldElement(self.x, self.curve.field)
        self.y = FieldElement(self.y, self.curve.field)

        # Verify if the point satisfies the curve equation
        if self not in self.curve:
            raise ValueError

    def __add__(self, other):
        if self == I:
            return other

        if other == I:
            return self

        if self.x == other.x and self.y == (-1 * other.y):
            return I

        if self.x != other.x:
            x1, x2 = self.x, other.x
            y1, y2 = self.y, other.y

            s = (y2 - y1) / (x2 - x1)
            x3 = s ** 2 - x1 - x2
            y3 = s * (x1 - x3) - y1

            return self.__class__(
                x=x3.value,
                y=y3.value,
                curve=secp256k1
            )

        if self == other and self.y == inf:
            return I

        if self == other:
            x1, y1, a = self.x, self.y, self.curve.a

            s = (3 * x1 ** 2 + a) / (2 * y1)
            x3 = s ** 2 - 2 * x1
            y3 = s * (x1 - x3) - y1

            return self.__class__(
                x=x3.value,
                y=y3.value,
                curve=secp256k1
            )
    
    def __rmul__(self, scalar: int) -> "Point":
        current = self
        result = I
        while scalar:
            if scalar & 1:  # same as scalar % 2
                result = result + current
            current = current + current  # point doubling
            scalar >>= 1  # same as scalar / 2
        return result


G = Point(
    x=0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    y=0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8,
    curve=secp256k1
)

# Order of the group generated by G, such that nG = I
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

I = Point(x=None, y=None, curve=secp256k1)

def public_key(private_key: int) -> Tuple[int, int]:
    K = private_key*G
    return K.x.value, K.y.value

def sha(a: str) -> str:
    return hashlib.sha256(a.encode('utf-8')).hexdigest()

def ripemd160(a: str) -> str:
    return hashlib.new('ripemd160', a.encode('utf-8')).hexdigest()

def raw_adress(public_key: str) -> str:
    # Here maybe accept Point and concat strings of both fields?
    s = ripemd160(sha(public_key))
    # return int(s, 16)
    return s

def base_58(num: int):
    """ Returns num in a base58-encoded string """
    encode = ''
    alphabet = '123456789abcdefghijkmnopqrstuvwxyzABCDEFGHJKLMNPQRSTUVWXYZ'
    base_count = len(alphabet)

    num = int(num, 16)
    if (num < 0):
        return ''

    while (num >= base_count):	
        mod = int(num % base_count)
        encode = alphabet[mod] + encode
        num = num // base_count

    if (num):
        encode = alphabet[num] + encode

    return encode

def base_58_check(raw_adress: str) -> str:
    prefix = '00'
    a = prefix + raw_adress
    checksum = sha(sha(a))[-4:]
    a += checksum
    return base_58(a)

In [13]:
# private_key = sha('something_random')
private_key = '0x1E99423A4ED27608A15A2616A2B0E9E52CED330AC530EDCC32C8FFC6A526AEDD'
print(f"private key (hex): {private_key}")
# print(f"private key (int): {int(private_key, 16)}")
_public_key = public_key(int(private_key, 16))
# print(f"public key x (int) {_public_key[0]}")
# print(f"public key y (int) {_public_key[1]}")
print(f"public key x (hex) {hex(_public_key[0])}")
print(f"public key y (hex) {hex(_public_key[1])}")

private key (hex): 0x1E99423A4ED27608A15A2616A2B0E9E52CED330AC530EDCC32C8FFC6A526AEDD
public key x (hex) 0xf028892bad7ed57d2fb57bf33081d5cfcf6f9ed3d3d7f159c2e2fff579dc341a
public key y (hex) 0x7cf33da18bd734c600b96a72bbc4749d5141c90ec8ac328ae52ddfe2e505bdb


In [22]:
code_strings = {
    2: '01',
    10: '0123456789',
    16: '0123456789abcdef',
    32: 'abcdefghijklmnopqrstuvwxyz234567',
    58: '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz',
    256: ''.join([chr(x) for x in range(256)])
}

def get_code_string(base):
    if base not in code_strings:
        raise ValueError("Invalid base!")
    return code_strings[base]

def encode(val, base, minlen=0):
    base, minlen = int(base), int(minlen)
    code_string = get_code_string(base)
    result_bytes = bytes()
    while val > 0:
        curcode = code_string[val % base]
        result_bytes = bytes([ord(curcode)]) + result_bytes
        val //= base

    pad_size = minlen - len(result_bytes)

    padding_element = b'\x00' if base == 256 else b'1' \
        if base == 58 else b'0'
    if (pad_size > 0):
        result_bytes = padding_element*pad_size + result_bytes

    result_string = ''.join([chr(y) for y in result_bytes])
    result = result_bytes if base == 256 else result_string

    return result

def encode_pubkey(pub: Tuple[int, int], formt: str):
    if formt == 'decimal': return pub
    elif formt == 'bin': return b'\x04' + encode(pub[0], 256, 32) + encode(pub[1], 256, 32)
    elif formt == 'hex': return '04' + encode(pub[0], 16, 64) + encode(pub[1], 16, 64)

def from_int_to_byte(a):
        return bytes([a])

def from_string_to_bytes(a):
        return a if isinstance(a, bytes) else bytes(a, 'utf-8')

def bin_dbl_sha256(s):
        bytes_to_hash = from_string_to_bytes(s)
        return hashlib.sha256(hashlib.sha256(bytes_to_hash).digest()).digest()

def lpad(msg, symbol, length):
        if len(msg) >= length:
            return msg
        return symbol * (length - len(msg)) + msg

def decode(string, base):
        if base == 256 and isinstance(string, str):
            string = bytes(bytearray.fromhex(string))
        base = int(base)
        code_string = get_code_string(base)
        result = 0
        if base == 256:
            def extract(d, cs):
                return d
        else:
            def extract(d, cs):
                return cs.find(d if isinstance(d, str) else chr(d))

        if base == 16:
            string = string.lower()
        while len(string) > 0:
            result *= base
            result += extract(string[0], code_string)
            string = string[1:]
        return result


def changebase(string, frm, to, minlen=0):
        if frm == to:
            return lpad(string, get_code_string(frm)[0], minlen)
        return encode(decode(string, frm), to, minlen)


def bin_to_b58check(inp, magicbyte=0):
        if magicbyte == 0:
            inp = from_int_to_byte(0) + inp
        while magicbyte > 0:
            inp = from_int_to_byte(magicbyte % 256) + inp
            magicbyte //= 256

        leadingzbytes = 0
        for x in inp:
            if x != 0:
                break
            leadingzbytes += 1

        checksum = bin_dbl_sha256(inp)[:4]
        return '1' * leadingzbytes + changebase(inp+checksum, 256, 58)

from ripemd import RIPEMD160
import binascii

def bin_hash160(string):
    intermed = hashlib.sha256(string).digest()
    digest = ''
    try:
        digest = hashlib.new('ripemd160', intermed).digest()
    except:
        digest = RIPEMD160(intermed).digest()
    return digest

def pubkey_to_address(pubkey, magicbyte=0):
    if isinstance(pubkey, (list, tuple)):
        pubkey = encode_pubkey(pubkey, 'bin')
    if len(pubkey) in [66, 130]:
        return bin_to_b58check(
            bin_hash160(binascii.unhexlify(pubkey)), magicbyte)
    return bin_to_b58check(bin_hash160(pubkey), magicbyte)

def privkey_to_address(priv, magicbyte=0):
    return pubkey_to_address(public_key(priv), magicbyte)
privtoaddr = privkey_to_address

In [23]:
# private_key = sha('something_random')
private_key = '0x038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776'
print(f"private key (hex): {private_key}")
# print(f"private key (int): {int(private_key, 16)}")
_public_key = public_key(int(private_key, 16))
# print(f"public key x (int) {_public_key[0]}")
# print(f"public key y (int) {_public_key[1]}")
print(f"public key x (hex) {hex(_public_key[0])}")
print(f"public key y (hex) {hex(_public_key[1])}")

private key (hex): 0x038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776
public key x (hex) 0x2a406624211f2abbdc68da3df929f938c3399dd79fac1b51b0e4ad1d26a47aa
public key y (hex) 0x9f3bc9f3948a19dabb796a2a744aae50367ce38a3e6b60ae7d72159caeb0c102


In [34]:
pubkey_to_address((_public_key[0], _public_key[1]))

'14K1y4Epb341duzDmWsPniLyBh9EVh8jG3'

In [24]:
encode_pubkey((_public_key[0], _public_key[1]), 'hex')

'0402a406624211f2abbdc68da3df929f938c3399dd79fac1b51b0e4ad1d26a47aa9f3bc9f3948a19dabb796a2a744aae50367ce38a3e6b60ae7d72159caeb0c102'

In [ ]:
class Wallet():
    def __init__(self, name: str = "mlem", amount: float = 0):
        self.name = name
        self.amount = amount
        self.utxo = []
        
        # random_seed = random.randint(0, 1000)
        # self.private_key = hash_unicode(str(random_seed))
        # self.private_key = 0xA0DC65FFCA799873CBEA0AC274015B9526505DAAAED385155425F7337704883E #find a way how to generate this
        # self.private_key = f"0x{hash_unicode(str(random_seed))}"
        # self.private_key_bin = ''.join(format(ord(x), 'b') for x in self.private_key)
        # self.private_key_int = int(self.private_key_bin, base=2)

    def take_amount(self, amount):
        self.amount -= amount

    def add_amount(self, amount):
        self.amount += amount

    def get_amount(self) -> float:
        return self.amount + sum(utxo_.amount for utxo_ in self.utxo)

    def __repr__(self) -> str:
        return f"Wallet: \n\tname: {self.name}, \n\tamount: {self.get_amount()}"

    def hash256(self) -> str:
        return hash_unicode(repr(self))